# Exercise: MA(1) Model

In [3]:
using DSGE
using DataFrames, DataStructures, Distributions, Plots
include("util.jl")

# This file defines the MA1 type
include("ma1.jl")

INFO: Precompiling module FixedSizeArrays.
INFO: Precompiling module RecipesBase.
INFO: Precompiling module PlotUtils.
INFO: Precompiling module PlotThemes.
INFO: Precompiling module Showoff.


In [4]:
# See docstring for MA1
?MA1

search: means_bands_all meansbands_matrix_all @test_matrix_approx_eq



```
MA1{T} <: AbstractModel{T}
```

Implements the following MA(1) model:

```
x_t = μ + u_t + α*u_{t-1}
u_t ∼ N(0, σ^2)
```


In [6]:
# Initialize MA(1) model
m = MA1()

m <= Setting(:saveroot, dirname(@__FILE__))
m <= Setting(:date_presample_start, DSGE.quartertodate("2001-Q1"))
m <= Setting(:date_mainsample_start, DSGE.quartertodate("2001-Q1"))
m <= Setting(:date_forecast_start, DSGE.quartertodate("2026-Q1"))
m <= Setting(:n_mh_simulations, 500)
m <= Setting(:n_mh_blocks, 10)
m <= Setting(:n_mh_burn, 2)

In [7]:
m.parameters

3-element Array{DSGE.AbstractParameter{Float64},1}:
 DSGE.UnscaledParameter{Float64,DSGE.Exponential}
(:μ)
μ: constant coefficient
LaTeX label: \mu
-----------------------------
unscaled, untransformed value:        +0.500000
prior distribution:
	Distributions.Normal{Float64}(μ=0.0, σ=1.0)
transformation for csminwel:
	x -> b + (1/c) * log(x-a)
parameter is not fixed
    
 DSGE.UnscaledParameter{Float64,DSGE.Exponential}
(:β)
β: coefficient on u_{t-1}
LaTeX label: \beta
-----------------------------
unscaled, untransformed value:        +0.500000
prior distribution:
	Distributions.Normal{Float64}(μ=0.0, σ=1.0)
transformation for csminwel:
	x -> b + (1/c) * log(x-a)
parameter is not fixed

 DSGE.UnscaledParameter{Float64,DSGE.Exponential}
(:σ)
σ: standard deviation of u_t
LaTeX label: \sigma
-----------------------------
unscaled, untransformed value:        +0.100000
prior distribution:
	DSGE.RootInverseGamma(ν=5.0, τ=0.5)
transformation for csminwel:
	x -> b + (1/c) * log(x-a)
paramet

In [8]:
m.endogenous_states

DataStructures.OrderedDict{Symbol,Int64} with 2 entries:
  :u_t  => 1
  :u_t1 => 2

In [9]:
m.exogenous_shocks

DataStructures.OrderedDict{Symbol,Int64} with 1 entry:
  :u_t => 1

In [10]:
m.observables

DataStructures.OrderedDict{Symbol,Int64} with 1 entry:
  :x_t => 1

## Exercise 1

Given the following definition of `eqcond` (which returns the equilibrium condition matrices for an `MA1` model), fill in the `ZZ` and `QQ` matrices in `measurement`.

In [11]:
function DSGE.eqcond(m::MA1)
    endo = m.endogenous_states
    exo  = m.exogenous_shocks
    eq   = m.equilibrium_conditions

    Γ0 = zeros(n_states(m), n_states(m))
    Γ1 = zeros(n_states(m), n_states(m))
    C  = zeros(n_states(m))
    Ψ  = zeros(n_states(m), n_shocks_exogenous(m))
    Π  = zeros(n_states(m), n_shocks_expectational(m))

    # Γ0*s_t = Γ1*s_{t-1} + Ψ*ϵ_t + Π*η_t + C
    # s_t = [u_t, u_{t-1}]'
    # ϵ_t = [u_t]'

    # Row 1: u_t = 0*u_{t-1} + 0*u_{t-2} + u_t
    Γ0[eq[:eq_u_t], endo[:u_t]] = 1
     Ψ[eq[:eq_u_t],  exo[:u_t]] = 1

    # Row 2: u_{t-1} = u_{t-1} + 0 u_{t-2} + 0 u_t
    Γ0[eq[:eq_u_t1], endo[:u_t1]] = 1
    Γ1[eq[:eq_u_t1], endo[:u_t]]  = 1

    return Γ0, Γ1, C, Ψ, Π
end

In [12]:
function DSGE.measurement{T<:AbstractFloat}(m::MA1{T}, TTT::Matrix{T}, RRR::Matrix{T}, CCC::Vector{T};
                                            shocks::Bool = false)
    endo = m.endogenous_states # OrderedDict{Symbol, Int} mapping state names (e.g. `:u_t`) to indices
    exo  = m.exogenous_shocks  # ... mapping shock names to indices
    obs  = m.observables       # ... mapping observable names to indices

    ZZ = zeros(n_observables(m), n_states(m))
    DD = zeros(n_observables(m))
    MM = zeros(n_observables(m), n_shocks_exogenous(m))
    EE = zeros(n_observables(m), n_observables(m))
    QQ = zeros(n_shocks_exogenous(m), n_shocks_exogenous(m))

    # y_t = Z*s_t + D
    # y_t = [x_t]'
    # s_t = [u_t, u_{t-1}]'

    # TODO: fill in entries of ZZ matrix
    # x_t = μ + u_t + β*u_{t-1}
    # DD[obs[:x_t]] = m[:μ]
    # ZZ[obs[:x_t], ...] = ...

    DD[obs[:x_t]] = m[:μ]
    ZZ[obs[:x_t], endo[:u_t]]  = 1
    ZZ[obs[:x_t], endo[:u_t1]] = m[:β]

    # TODO: fill in entries of QQ matrix
    # QQ[exo[:u_t], exo[:u_t]] = ...

    QQ[exo[:u_t], exo[:u_t]] = m[:σ]^2

    HH    = EE + MM*QQ*MM'
    VV    = QQ*MM'
    VVall = [[RRR*QQ*RRR' RRR*VV];
             [VV'*RRR'    HH]]

    return Measurement(ZZ, DD, QQ, EE, MM, VVall)
end

## Exercise 2

Generate 4000 draws from the prior distribution of each parameter.

In [19]:
# Hint: use the `rand` function, which can be called on any `Distribution`.
dist_μ = get(m[:μ].prior)
dist_β = get(m[:β].prior)
dist_σ = get(m[:σ].prior)

isa(dist_μ, Distribution)

true

In [21]:
prior_draws = zeros(4000, 3)
for i = 1:4000
    # TODO
    prior_draws[i, 1] = rand(dist_μ)
    prior_draws[i, 2] = rand(dist_β)
    prior_draws[i, 3] = rand(dist_σ)
end

In [32]:
# Plot histograms of prior draws
prior_plot = plot(prior_draws, 
                  layout = @layout([a b c]),
                  label = [m[:μ].tex_label m[:β].tex_label m[:σ].tex_label],
                  t     = [:histogram      :histogram      :histogram],
                  color = [:red            :blue           :green],
                  title = ["" "Prior" ""])

INFO: binning = auto
INFO: binning = auto
INFO: binning = auto


## Exercise 3

Given data generated below, sample from the posterior distribution of the parameters. (No new code needs to be written in this section.)

In [35]:
# True values of parameters
μ = 0.75
β = 0.9
σ = 0.25

dist_u = Normal(0, σ)

# Initialize states
u_t  = 0.0
u_t1 = 0.0

# Initialize DataFrame with 100 periods (2001-Q1 to 2025-Q4)
df = DataFrame(date = DSGE.quarter_range(DSGE.quartertodate("2001-Q1"), DSGE.quartertodate("2025-Q4")))
df[:x_t] = NaN

for t = 1:100
    # Set last period's u_t value to this period's u_{t-1}
    u_t1 = u_t

    # Draw new value of u_t
    u_t = rand(dist_u)

    # Apply measurement equation to get x_t
    x_t = μ + u_t + β*u_t1

    # Record in DataFrame
    df[t, :x_t] = x_t
end

1×100 Array{Float64,2}:
 0.921924  0.899823  0.850224  0.725664  …  1.3052  1.24198  1.10452  1.02137

In [36]:
# Find posterior mode and sample using Metropolis-Hastings
# This function call will take a minute to run the first time
estimate(m, df)

Reoptimizing...
Iter     Function value   Gradient norm 
     0     3.428229e+02     8.779218e+02
     1     1.041688e+02     3.309445e+02
     2     2.070961e+01     8.532763e+01
     3     1.110817e+01     3.589797e+01
     4     1.043034e+01     3.846018e+01
     5     8.395198e+00     1.600176e+01
     6     7.189949e+00     1.644132e+01
     7     7.109600e+00     1.582007e+01
     8     6.898947e+00     1.442568e+01
     9     4.157852e+00     1.454391e+01
    10     3.621423e+00     8.582457e+00
    11     3.439271e+00     3.847890e+00
    12     3.395419e+00     8.416421e-01
    13     3.392303e+00     4.397252e-02
    14     3.392294e+00     3.587564e-03
    15     3.392294e+00     3.367704e-04
    16     3.392294e+00     7.744492e-08
improvement < ftol -- terminating
    17     3.392294e+00     5.345125e-07
Total iterations completed: 17
Optimization time elapsed:  9.76
Recalculating Hessian...
Hessian element: (1, 1)
Hessian element: (2, 2)
Hessian element: (3, 3)
Hessian el

In [61]:
# Load posterior draws
post_draws = load_draws(m, :full)

# Plot posterior draws
post_plot = plot(post_draws,
                 layout = @layout([a b c]),
                 label = [m[:μ].tex_label m[:β].tex_label m[:σ].tex_label],
                 t     = [:histogram      :histogram      :histogram],
                 color = [:red            :blue           :green],
                 title = ["" "Posterior" ""])

# Plot actual values
plot_actual!(post_plot.subplots[1], μ)
plot_actual!(post_plot.subplots[2], β)
plot_actual!(post_plot.subplots[3], σ)

Loading draws from output_data/ma1/ss0/estimate/raw/mhsave_vint=170626.h5


INFO: binning = auto
INFO: binning = auto
INFO: binning = auto


In [63]:
# Plot actuals on top of prior
plot_actual!(prior_plot.subplots[1], μ)
plot_actual!(prior_plot.subplots[2], β)
plot_actual!(prior_plot.subplots[3], σ)

# Plot both prior and posterior together
all_plots = plot(prior_plot, post_plot, layout = @layout([a; b]))